# Bottom-Up NeuralProphet Submission Generator

Streamlined notebook for generating Kaggle submissions using NeuralProphet.

**Strategy**: Train individual models for stores 1-10, then sum predictions for Store 0 (bottom-up approach).

## 01 - Imports and Config

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from pathlib import Path
from neuralprophet import NeuralProphet
import warnings
warnings.filterwarnings('ignore')

from src.data_loader import load_all_data, parse_submission_id

pd.set_option('display.max_columns', None)

# =============================================================================
# CONFIGURATION - Adjust these values to tune the model
# =============================================================================
EPOCHS = 200          # Try: 200, 300, 500 for better convergence
N_LAGS = 84           # Try: 56 (8 weeks), 84 (12 weeks) for more history
LEARNING_RATE = 0.005  # Try: 0.005, 0.001 for more stable training

# Future regressors to use
FUTURE_REGRESSORS = [
    'is_major_holiday',
    'is_sports_event',
    'is_religious_event',
    'month',
    'is_weekend'
]

print(f"Configuration: epochs={EPOCHS}, n_lags={N_LAGS}, lr={LEARNING_RATE}")

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


Configuration: epochs=200, n_lags=84, lr=0.005


## 02 - Load Data

In [2]:
# Load data
train_df, calendar_df, submission_df = load_all_data()

print(f"Training data shape: {train_df.shape}")
print(f"Date range: {train_df['date'].min()} to {train_df['date'].max()}")
print(f"Stores: {sorted(train_df['store_id'].unique())}")
print(f"\nCalendar events shape: {calendar_df.shape}")

Training data shape: (18766, 4)
Date range: 2011-01-29 00:00:00 to 2015-09-30 00:00:00
Stores: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Calendar events shape: (162, 2)


## 03 - Feature Engineering

In [3]:
def prepare_calendar_features(df, calendar_df):
    """
    Add calendar event features to dataframe.
    """
    df = df.copy()
    df['month'] = df['date'].dt.month
    df['is_weekend'] = df['date'].dt.dayofweek.isin([5, 6]).astype(int)
    df['day_of_week'] = df['date'].dt.dayofweek

    # Parse calendar dates (DD/MM/YYYY format)
    calendar_df = calendar_df.copy()
    calendar_df['date'] = pd.to_datetime(calendar_df['date'], format='%d/%m/%Y')
    
    # Define event categories
    major_holidays = ['Christmas', 'Thanksgiving', 'NewYear', 'IndependenceDay', 
                      'MemorialDay', 'LaborDay', 'ValentinesDay', "Mother's day", 
                      "Father's day"]
    sports_events = ['SuperBowl', 'NBAFinalsStart', 'NBAFinalsEnd']
    religious_events = ['Easter', 'OrthodoxEaster', 'LentStart', 'Ramadan starts', 
                        'Eid al-Fitr', 'Pesach End']
    
    # Initialize event columns
    df['is_major_holiday'] = 0
    df['is_sports_event'] = 0
    df['is_religious_event'] = 0
    
    # Mark events
    for _, row in calendar_df.iterrows():
        event_date = row['date']
        events = [e.strip() for e in str(row['event']).split(',')]
        
        mask = df['date'] == event_date
        
        for event in events:
            if event in major_holidays:
                df.loc[mask, 'is_major_holiday'] = 1
            if event in sports_events:
                df.loc[mask, 'is_sports_event'] = 1
            if event in religious_events:
                df.loc[mask, 'is_religious_event'] = 1
    
    return df


def prepare_neuralprophet_data(df, store_id):
    """
    Prepare data for NeuralProphet format.
    """
    store_df = df[df['store_id'] == store_id].copy()
    store_df = store_df.sort_values('date').reset_index(drop=True)
    
    prophet_df = pd.DataFrame({
        'ds': store_df['date'],
        'y': np.log1p(store_df['revenue'])
    })
    
    for col in FUTURE_REGRESSORS:
        if col in store_df.columns:
            prophet_df[col] = store_df[col].values
    
    return prophet_df


# Apply calendar features to full training data
full_train_with_events = prepare_calendar_features(train_df, calendar_df)
print(f"Training data with features: {full_train_with_events.shape}")
print(f"Columns: {list(full_train_with_events.columns)}")

Training data with features: (18766, 10)
Columns: ['store_id', 'store_name', 'date', 'revenue', 'month', 'is_weekend', 'day_of_week', 'is_major_holiday', 'is_sports_event', 'is_religious_event']


## 04 - Model Helpers

In [4]:
def train_neuralprophet_model(train_df, store_id, forecast_horizon, add_regressors=True, verbose=False,
                               epochs=100, n_lags=28, learning_rate=0.01):
    """
    Train NeuralProphet model for a specific store.
    """
    df = prepare_neuralprophet_data(train_df, store_id)
    
    model = NeuralProphet(
        growth='linear',
        n_changepoints=10,
        changepoints_range=0.9,
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode='additive',
        seasonality_reg=0.1,
        n_forecasts=forecast_horizon,                    
        n_lags=n_lags,
        ar_layers=[32, 32],
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=32,
        normalize='auto',
        drop_missing=True,
        trainer_config={
            'logger': False,
            'enable_progress_bar': verbose,
        }
    )
    
    if add_regressors:
        for col in FUTURE_REGRESSORS:
            model.add_future_regressor(col)
    
    model.fit(df, freq='D', validation_df=None, progress=None if not verbose else 'bar')
    model.forecast_horizon = forecast_horizon
    
    return model, df


def predict_neuralprophet(model, train_df, holdout_df, store_id):
    """
    Make predictions using trained NeuralProphet model.
    """
    train_prophet = prepare_neuralprophet_data(train_df, store_id)

    store_future = holdout_df[holdout_df['store_id'] == store_id].copy()
    store_future = store_future.sort_values('date').reset_index(drop=True)
    n_future = len(store_future)

    future_rows = pd.DataFrame({
        'ds': store_future['date'].values,
        'y': [None] * n_future
    })

    for col in FUTURE_REGRESSORS:
        if col in store_future.columns:
            future_rows[col] = store_future[col].values
        else:
            future_rows[col] = 0

    combined_df = pd.concat([train_prophet, future_rows], ignore_index=True)
    forecast = model.predict(combined_df)

    train_end_date = train_prophet['ds'].max()
    future_forecast = forecast[forecast['ds'] > train_end_date].reset_index(drop=True)

    if 'yhat1' in future_forecast.columns:
        predictions = []
        for i in range(min(n_future, len(future_forecast))):
            col_name = f'yhat{i+1}'
            if col_name in future_forecast.columns and i < len(future_forecast):
                predictions.append(future_forecast.loc[i, col_name])
            elif 'yhat1' in future_forecast.columns:
                predictions.append(future_forecast.loc[i, 'yhat1'])
        predictions = np.array(predictions, dtype=float)
    else:
        predictions = future_forecast['yhat'].values

    predictions = np.expm1(predictions)

    if len(predictions) < n_future:
        last_val = predictions[-1] if len(predictions) > 0 else 0
        predictions = np.concatenate([predictions, [last_val] * (n_future - len(predictions))])
    predictions = predictions[:n_future]

    return predictions


print("Model helper functions defined.")

Model helper functions defined.


## 05 - Prepare Submission Dates

In [5]:
# Parse submission dates
submission_dates = []
for _, row in submission_df.iterrows():
    store_id, date_str = parse_submission_id(row['id'])
    date = pd.to_datetime(date_str, format='%Y%m%d')
    submission_dates.append({
        'id': row['id'],
        'store_id': store_id,
        'date': date
    })

submission_forecast_df = pd.DataFrame(submission_dates)
submission_forecast_df = prepare_calendar_features(submission_forecast_df, calendar_df)

SUBMISSION_HORIZON = submission_forecast_df['date'].nunique()

print(f"Forecast period: {submission_forecast_df['date'].min()} to {submission_forecast_df['date'].max()}")
print(f"Forecast horizon: {SUBMISSION_HORIZON} days")
print(f"Total predictions needed: {len(submission_forecast_df)}")

Forecast period: 2015-10-01 00:00:00 to 2015-12-31 00:00:00
Forecast horizon: 92 days
Total predictions needed: 1012


## 06 - Train Final Models

In [6]:
print(f"Training NeuralProphet models on full dataset...")
print(f"Using: epochs={EPOCHS}, n_lags={N_LAGS}, lr={LEARNING_RATE}")
print("="*60)

final_models = {}

for store_id in range(1, 11):
    store_name = train_df[train_df['store_id'] == store_id]['store_name'].iloc[0]
    print(f"\nTraining Store {store_id}: {store_name}...")
    
    model, _ = train_neuralprophet_model(
        full_train_with_events, 
        store_id,
        forecast_horizon=SUBMISSION_HORIZON,
        add_regressors=True,
        epochs=EPOCHS,
        n_lags=N_LAGS,
        learning_rate=LEARNING_RATE
    )
    final_models[store_id] = model
    print(f"  Done.")

print("\n" + "="*60)
print(f"All {len(final_models)} models trained successfully!")

INFO - (NP.config.__post_init__) - Note: Fourier-based seasonality regularization is experimental.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.941% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.


Training NeuralProphet models on full dataset...
Using: epochs=200, n_lags=84, lr=0.005

Training Store 1: California – Sunset Plaza...


INFO - (NP.config.__post_init__) - Note: Fourier-based seasonality regularization is experimental.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.941% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.


  Done.

Training Store 2: California – Ocean View...


INFO - (NP.config.__post_init__) - Note: Fourier-based seasonality regularization is experimental.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.941% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.


...

  Done.

Training Store 8: Wisconsin – Maple Grove...


INFO - (NP.config.__post_init__) - Note: Fourier-based seasonality regularization is experimental.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.941% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.


  Done.

Training Store 9: Wisconsin – Lakeview Plaza...


INFO - (NP.config.__post_init__) - Note: Fourier-based seasonality regularization is experimental.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.941% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.


  Done.

Training Store 10: Wisconsin – Badger Crossing...
  Done.

All 10 models trained successfully!


## 07 - Generate Predictions

In [7]:
print("Generating predictions for stores 1-10...")
print("="*60)

final_predictions = []

for store_id in range(1, 11):
    store_name = train_df[train_df['store_id'] == store_id]['store_name'].iloc[0]
    print(f"\nPredicting Store {store_id}: {store_name}...")
    
    model = final_models[store_id]
    preds = predict_neuralprophet(model, full_train_with_events, submission_forecast_df, store_id)
    
    store_submission = submission_forecast_df[submission_forecast_df['store_id'] == store_id].copy()
    store_submission = store_submission.sort_values('date').reset_index(drop=True)
    store_submission['prediction'] = preds
    final_predictions.append(store_submission[['id', 'prediction']])
    
    print(f"  Generated {len(preds)} predictions")
    print(f"  Mean: {preds.mean():,.2f}, Min: {preds.min():,.2f}, Max: {preds.max():,.2f}")

final_submission = pd.concat(final_predictions, ignore_index=True)

print("\n" + "="*60)
print(f"Generated {len(final_submission)} predictions for stores 1-10")

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.944% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.944% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.data.processing._handle_missing_data) - Dropped 92 rows at the end with NaNs in 'y' column.


Generating predictions for stores 1-10...

Predicting Store 1: California – Sunset Plaza...


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.944% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.944% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.data.processing._handle_missing_data) - Dropped 92 rows at the end with NaNs in 'y' column.


  Generated 92 predictions
  Mean: 33,376.05, Min: 16,506.73, Max: 45,685.32

Predicting Store 2: California – Ocean View...


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.944% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.944% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.data.processing._handle_missing_data) - Dropped 92 rows at the end with NaNs in 'y' column.


...

  Generated 92 predictions
  Mean: 22,277.91, Min: 12,444.14, Max: 33,212.55

Predicting Store 9: Wisconsin – Lakeview Plaza...


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.944% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.944% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.data.processing._handle_missing_data) - Dropped 92 rows at the end with NaNs in 'y' column.


  Generated 92 predictions
  Mean: 25,954.56, Min: 11,965.93, Max: 32,259.15

Predicting Store 10: Wisconsin – Badger Crossing...


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


  Generated 92 predictions
  Mean: 20,659.43, Min: 10,796.20, Max: 29,391.28

Generated 920 predictions for stores 1-10


## 08 - Store 0 Aggregation (Bottom-Up)

In [8]:
print("Creating Store 0 predictions by summing stores 1-10...")

store_1_10_preds = submission_forecast_df[submission_forecast_df['store_id'] != 0].copy()
store_1_10_preds = store_1_10_preds.merge(
    final_submission[['id', 'prediction']],
    on='id',
    how='left'
)

store_0_preds = store_1_10_preds.groupby('date')['prediction'].sum().reset_index()
store_0_preds['store_id'] = 0
store_0_preds['id'] = store_0_preds['date'].apply(
    lambda d: f"0_{d.strftime('%Y%m%d')}"
)

final_submission = pd.concat([
    store_0_preds[['id', 'prediction']],
    final_submission,
], ignore_index=True)

final_submission['prediction'] = final_submission['prediction'].clip(lower=0)

print(f"\nStore 0 predictions:")
print(f"  Mean: {store_0_preds['prediction'].mean():,.2f}")
print(f"  Min: {store_0_preds['prediction'].min():,.2f}")
print(f"  Max: {store_0_preds['prediction'].max():,.2f}")
print(f"\nTotal submission size: {len(final_submission)}")

Creating Store 0 predictions by summing stores 1-10...

Store 0 predictions:
  Mean: 280,970.11
  Min: 150,085.80
  Max: 361,117.21

Total submission size: 1012


## 09 - Save Submission

In [9]:
submissions_dir = Path('../submissions/neuralprophet')
submissions_dir.mkdir(parents=True, exist_ok=True)

from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_path = submissions_dir / f'bottom_up_neuralprophet_{timestamp}.csv'

final_submission.to_csv(output_path, index=False)

print(f"\n" + "="*60)
print(f"SUBMISSION SAVED")
print(f"="*60)
print(f"\nFile: {output_path}")
print(f"\nSummary:")
print(f"  Total predictions: {len(final_submission)}")
print(f"  Mean prediction: {final_submission['prediction'].mean():,.2f}")
print(f"  Min prediction: {final_submission['prediction'].min():,.2f}")
print(f"  Max prediction: {final_submission['prediction'].max():,.2f}")
print(f"\nFirst 5 rows:")
print(final_submission.head())
print(f"\nLast 5 rows:")
print(final_submission.tail())


SUBMISSION SAVED

File: ../submissions/neuralprophet/bottom_up_neuralprophet_20260103_224013.csv

Summary:
  Total predictions: 1012
  Mean prediction: 51,085.47
  Min prediction: 8,904.21
  Max prediction: 361,117.21

First 5 rows:
           id     prediction
0  0_20151001  258686.431560
1  0_20151002  295067.523183
2  0_20151003  352825.673482
3  0_20151004  358673.183478
4  0_20151005  290387.832959

Last 5 rows:
               id    prediction
1007  10_20151227  21643.336867
1008  10_20151228  17923.869029
1009  10_20151229  15671.774324
1010  10_20151230  14870.778711
1011  10_20151231  16386.375785
